# EE 599 HW 3 Part 1: Kernel Design

Your task in this Colab notebook is to fill out the sections that are specified by **TODO** (please search the keyword `TODO` to make sure you do not miss any).

## Im2col Algorithm


`Im2col` is a method used in CNNs to transform the input data and filters into a format that allows the convolution to be expressed as a matrix multiplication. This transformation can simplify the implementation of convolution and leverage highly optimized matrix multiplication routines such as BLAS library.

In the class and discussion, we have covered the `im2col` algorithm for 2D input with 2D filter.

In this section, we extend and implement the `im2col` algorithm for 4D input with 4D filters.

First, let's import packages, configure the convolution operation, and randomly initialize the input and filters.

In [66]:
import torch
import torch.nn.functional as F

padding = 0
stride = 1

N, C, H, W = 4, 3, 5, 5
K, C, KH, KW = 2, 3, 3, 3

torch.manual_seed(10) # assign the random seed to avoid precision issue

input = torch.rand(N, C, H, W)
filter = torch.randn(K, C, KH, KW)

### **TODO 1:**

Calculate the output height `OH` and output width `OW` for the convolution operation, based on the input dimensions, padding, stride, and kernel size.

In [44]:
#TODO: compute OH and OW
OH = (H - KH + stride + 2*padding) // stride
OW = (W - KW + stride + 2*padding) // stride

### **TODO 2**:

Implement the `im2col` operation to transform the 4D input and filter tensors into two 2D matrices. After matrix multiplication, reshape the result back into a 4D tensor to simulate the convolution operation. Compare your implementation's result `output_im2col` with PyTorch's `conv2d` function to verify correctness.

In [46]:
#TODO: implement im2col for 4D input and filter

filter_2d = torch.zeros((K, C*KH*KW))
for k in range(K):
  for c in range(C):
    filter_2d[k, c*KH*KW:(c+1)*KH*KW] = filter[k, c, :, :].flatten()

input_2d = torch.zeros((C*KH*KW, N*OH*OW))

for n in range(N):
  for c in range(C):
    col = 0
    for i in range(OH):
      for j in range(OW):
        input_2d[c*KH*KW:(c+1)*KH*KW, n*OW*OH+col] = input[n, c, i:i+KH, j:j+KW].flatten()
        col += 1

output_2d = torch.mm(filter_2d, input_2d)

output_im2col = torch.zeros(N, K, OH, OW)
for n in range(N):
  for k in range(K):
    for i in range(OH):
      for j in range(OW):
        output_im2col[n, k, i, j] = output_2d[k, n*OH*OW+i*OW+j]

print(output_im2col)

tensor([[[[ 1.8768,  3.0835,  1.9521],
          [ 3.9751,  0.6928,  2.5260],
          [-1.6710,  0.9018,  2.2751]],

         [[-2.5823, -3.2104, -2.1703],
          [-3.6444, -1.6703, -2.5939],
          [-1.6377, -2.4205, -0.5754]]],


        [[[ 2.2483,  1.3231,  1.5150],
          [ 2.4206,  0.0365,  1.8823],
          [ 2.8809,  0.9102,  4.3302]],

         [[-0.8055, -0.9987, -3.0501],
          [-1.5590, -0.2290, -3.7441],
          [-0.5181, -3.8220, -2.3892]]],


        [[[-1.7639,  3.2723,  2.7127],
          [ 3.1597,  0.7641, -1.9370],
          [ 2.7992,  1.0212,  2.8657]],

         [[-0.6126, -4.8531, -2.5797],
          [-2.2863, -0.9493, -0.7335],
          [-3.0289, -2.2384, -1.0353]]],


        [[[ 2.9547,  2.8948,  1.8939],
          [ 4.0806,  3.7154,  3.8900],
          [ 3.8496,  0.8052,  0.8200]],

         [[-1.1793, -2.9636, -3.4071],
          [-2.4364, -5.4134, -3.1107],
          [-1.7426, -1.6712, -1.4955]]]])


The correct result is provided below by using Pytorch's `conv2d` function.

In [47]:
output_conv2d = F.conv2d(input, filter, stride=stride, padding=padding)
print(output_conv2d)

tensor([[[[ 1.8768,  3.0835,  1.9521],
          [ 3.9751,  0.6928,  2.5260],
          [-1.6710,  0.9018,  2.2751]],

         [[-2.5823, -3.2104, -2.1703],
          [-3.6444, -1.6703, -2.5939],
          [-1.6377, -2.4205, -0.5754]]],


        [[[ 2.2483,  1.3231,  1.5150],
          [ 2.4206,  0.0365,  1.8823],
          [ 2.8809,  0.9102,  4.3302]],

         [[-0.8055, -0.9987, -3.0501],
          [-1.5590, -0.2290, -3.7441],
          [-0.5181, -3.8220, -2.3892]]],


        [[[-1.7639,  3.2723,  2.7127],
          [ 3.1597,  0.7641, -1.9370],
          [ 2.7992,  1.0212,  2.8657]],

         [[-0.6126, -4.8531, -2.5797],
          [-2.2863, -0.9493, -0.7335],
          [-3.0289, -2.2384, -1.0353]]],


        [[[ 2.9547,  2.8948,  1.8939],
          [ 4.0806,  3.7154,  3.8900],
          [ 3.8496,  0.8052,  0.8200]],

         [[-1.1793, -2.9636, -3.4071],
          [-2.4364, -5.4134, -3.1107],
          [-1.7426, -1.6712, -1.4955]]]])


We can use the following functions to check how many the elements are matched.

In [48]:
# print sum of the True values
print("Number of matched elements =", torch.isclose(output_im2col, output_conv2d).sum().item())

# print the total number of values
print("Number of total elements =", output_im2col.numel())


Number of matched elements = 72
Number of total elements = 72


## Matrix Multiplication Optimization

While matrix multiplication operations are foundational, they can pose challenges, especially in terms of expensive computational complexity.
Present compilers are incapable of fully harnessing the processor architecture complexity. There is a wide gap between the available and achieved performance of software. Thereby, the need for performance tuning. Performance tuning of the simple matrix multiplication has indeed been a very tough and challenging project. In this section, we discuss some of the optimization techniques, which gave us substantial improvements.

To evaluate and improve the performance of matrix multiplication implementations, it's beneficial to use low-level programming languages like C or C++, which offer closer control over hardware resources. Within a notebook environment, we can facilitate the development, compilation, and execution of C code by using specific commands. The `%%writefile` command allows us to save the content of a notebook cell directly into a file, which can then be compiled and executed using command-line instructions.

In the cell below, we provide a naive matrix multiplication implementation and measure the FLOPs per section.

In [49]:
%%writefile naive_mm.c
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

int main(void) {
    int i, j, k;
    struct timespec start, stop;
    double time;
    int n = 1024; // Matrix size is n*n

    // Allocate memory for matrices A, B, and C
    double **A = (double**) malloc(sizeof(double*) * n);
    double **B = (double**) malloc(sizeof(double*) * n);
    double **C = (double**) malloc(sizeof(double*) * n);
    for (i = 0; i < n; i++) {
        A[i] = (double*) malloc(sizeof(double) * n);
        B[i] = (double*) malloc(sizeof(double) * n);
        C[i] = (double*) malloc(sizeof(double) * n);
    }

    // Initialize matrices A and B
    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i][j] = i;
            B[i][j] = i + j;
            C[i][j] = 0;
        }
    }

    // Start timer
    if (clock_gettime(CLOCK_REALTIME, &start) == -1) {
        perror("clock gettime");
    }

    // Naive Matrix Multiplication
    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            for (k = 0; k < n; k++) {
                C[i][j] = C[i][j] + A[i][k] * B[k][j];
            }
        }
    }

    // Stop timer
    if (clock_gettime(CLOCK_REALTIME, &stop) == -1) {
        perror("clock gettime");
    }
    time = (stop.tv_sec - start.tv_sec) + (double)(stop.tv_nsec - start.tv_nsec) / 1e9;

    // Print results
    printf("Number of FLOPs = %u, Execution time = %f sec,\n%lf MFLOPs per sec\n", 2 * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);
    printf("C[100][100]=%f\n", C[100][100]);

    // Release memory
    for (i = 0; i < n; i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
    }
    free(A);
    free(B);
    free(C);

    return 0;
}

Writing naive_mm.c


Compile and execute the code.

In [50]:
!g++ naive_mm.c -o naive_mm && ./naive_mm

Number of FLOPs = 2147483648, Execution time = 24.726726 sec,
86.848687 MFLOPs per sec
C[100][100]=62617600.000000


### **TODO 3:**

Blocked matrix multiplication, also known as tiled matrix multiplication, is an optimization technique used to improve the performance of matrix multiplication operations, especially on modern hardware with hierarchical memory systems. This approach involves dividing the input matrices into smaller sub-matrices or "blocks" and then performing the multiplication on these blocks rather than on individual elements.

In the cell below, fill out the missing code.

In [67]:
%%writefile blocking_mm.c
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

int main(int argc, char *argv[]) {
    int i, j, k, ii, jj, kk;
    struct timespec start, stop;
    double time;
    int n = 1024; // Matrix size is n*n
    int b = atoi(argv[1]); // Block size

    // Allocate memory for matrices A, B, and C
    double **A = (double**) malloc(sizeof(double*) * n);
    double **B = (double**) malloc(sizeof(double*) * n);
    double **C = (double**) malloc(sizeof(double*) * n);
    for (i = 0; i < n; i++) {
        A[i] = (double*) malloc(sizeof(double) * n);
        B[i] = (double*) malloc(sizeof(double) * n);
        C[i] = (double*) malloc(sizeof(double) * n);
    }

    // Initialize matrices A and B
    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i][j] = i;
            B[i][j] = i + j;
            C[i][j] = 0;
        }
    }

    // Start timer
    if (clock_gettime(CLOCK_REALTIME, &start) == -1) {
        perror("clock gettime");
    }

    // TODO: Blocking Matrix Multiplication
    // Your code goes here
    //*******************************//
    for (i=0; i<n; i+=b){
      for (j=0; j<n; j+=b){
        for (k=0; k<n; k+=b){
          for (ii=i; ii<i+b; ii++){
            for (jj=j; jj<j+b; jj++){
              for (kk=k; kk<k+b; kk++){
                C[ii][jj] += A[ii][kk] * B[kk][jj];
              }
            }
          }
        }
      }
    }
    //*******************************//

    // Stop timer
    if (clock_gettime(CLOCK_REALTIME, &stop) == -1) {
        perror("clock gettime");
    }
    time = (stop.tv_sec - start.tv_sec) + (double)(stop.tv_nsec - start.tv_nsec) / 1e9;

    // Print results
    printf("Number of FLOPs = %u, Execution time = %f sec,\n%lf MFLOPs per sec\n", 2 * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);
    printf("C[100][100]=%f\n", C[100][100]);

    // Release memory
    for (i = 0; i < n; i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
    }
    free(A);
    free(B);
    free(C);

    return 0;
}

Overwriting blocking_mm.c


Compile and execute the code with different block sizes.

In [54]:
!g++ blocking_mm.c -o blocking_mm && ./blocking_mm 4 && ./blocking_mm 8 && ./blocking_mm 16

Number of FLOPs = 2147483648, Execution time = 11.187395 sec,
191.955645 MFLOPs per sec
C[100][100]=62617600.000000
Number of FLOPs = 2147483648, Execution time = 9.132075 sec,
235.158351 MFLOPs per sec
C[100][100]=62617600.000000
Number of FLOPs = 2147483648, Execution time = 8.078163 sec,
265.838128 MFLOPs per sec
C[100][100]=62617600.000000


### **TODO 4:**

OpenMP is a powerful API designed for parallel programming in C, C++, and Fortran, enabling efficient utilization of multicore and multiprocessor systems. It simplifies the development of parallel applications by providing a set of straightforward compiler directives, library routines, and environment variables that abstract away the complexities of thread management and synchronization. By allowing code to be parallelized with minimal modifications, OpenMP fosters portability and scalability across various platforms.

In the cell below, use the proper pragma configuaration to execute your for loops in parallel. You need to make sure the index is a private variable to each thread, otherwise race conditions might happen. We use the default number of threads in Colab enviroment.

In [64]:
%%writefile blocking_mt_mm.c
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <omp.h>

int main(int argc, char *argv[]) {
    int i, j, k, ii, jj, kk;
    struct timespec start, stop;
    double time;
    int n = 1024; // Matrix size is n*n
    int b = atoi(argv[1]); // Block size

    // Allocate memory for matrices A, B, and C
    double **A = (double**) malloc(sizeof(double*) * n);
    double **B = (double**) malloc(sizeof(double*) * n);
    double **C = (double**) malloc(sizeof(double*) * n);
    for (i = 0; i < n; i++) {
        A[i] = (double*) malloc(sizeof(double) * n);
        B[i] = (double*) malloc(sizeof(double) * n);
        C[i] = (double*) malloc(sizeof(double) * n);
    }

    // Initialize matrices A and B
    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i][j] = i;
            B[i][j] = i + j;
            C[i][j] = 0;
        }
    }

    // Start timer
    if (clock_gettime(CLOCK_REALTIME, &start) == -1) {
        perror("clock gettime");
    }

    // TODO: Blocking Matrix Multiplication with OpenMP
    // Your code goes here
    //*******************************//
    #pragma omp parallel for private(i, j, k, ii, jj, kk) collapse(2)
    for (i=0; i<n; i+=b){
      for (j=0; j<n; j+=b){
        for (k=0; k<n; k+=b){
          for (ii=i; ii<i+b; ii++){
            for (jj=j; jj<j+b; jj++){
              for (kk=k; kk<k+b; kk++){
                C[ii][jj] += A[ii][kk] * B[kk][jj];
              }
            }
          }
        }
      }
    }
    //*******************************//

    // Stop timer
    if (clock_gettime(CLOCK_REALTIME, &stop) == -1) {
        perror("clock gettime");
    }
    time = (stop.tv_sec - start.tv_sec) + (double)(stop.tv_nsec - start.tv_nsec) / 1e9;

    // Print results
    printf("Number of FLOPs = %u, Execution time = %f sec,\n%lf MFLOPs per sec\n", 2 * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);
    printf("C[100][100]=%f\n", C[100][100]);

    // Release memory
    for (i = 0; i < n; i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
    }
    free(A);
    free(B);
    free(C);

    return 0;
}

Overwriting blocking_mt_mm.c


Compile and execute the code with different block sizes.

In [65]:
!g++ -fopenmp blocking_mt_mm.c -o blocking_mt_mm && ./blocking_mt_mm 4 && ./blocking_mt_mm 8 && ./blocking_mt_mm 16

Number of FLOPs = 2147483648, Execution time = 9.532337 sec,
225.284063 MFLOPs per sec
C[100][100]=62617600.000000
Number of FLOPs = 2147483648, Execution time = 6.832648 sec,
314.297440 MFLOPs per sec
C[100][100]=62617600.000000
Number of FLOPs = 2147483648, Execution time = 7.451108 sec,
288.209965 MFLOPs per sec
C[100][100]=62617600.000000
